In [ ]:
import pandas as pd
import os
import arcpy
from arcgis.gis import GIS
from arcgis.server import AdminAPI
from arcgis.mapping import MapImageLayer
from arcgis.features import FeatureLayerCollection

#### Get Data via API

In [ ]:
# API key and endpoint
api_url = ""
api_key = ""

#### Transform Data

In [ ]:
vru_approach_dict = {
    'NB': 'S',
    'SB': 'N',
    'EB': 'W',
    'WB': 'E'
}

class_dict = {
    'Mobility Aid': 'Pedestrian',
    'Motorcycle': 'Passenger Vehicle',
    'Articulated Truck': 'Semi Truck',
    'Single Unit Truck': 'Box Truck',
    'Person Mobility Device': 'Pedestrian'
}

severity_dict = {
    'High': 'Severe',
    'Low': 'Moderate'
}

default_date = '1900-01-01'

In [ ]:
def import_and_concatenate_csvs(folder_path):
    # List all CSV files in the folder
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    
    # Initialize an empty list to store DataFrames
    df_list = []

    # Iterate through each CSV file
    for file in csv_files:
        # Construct the full file path
        file_path = os.path.join(folder_path, file)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        df_list.append(df)
        concatenated_df = pd.concat(df_list, ignore_index=True)
        # pull the location from the file name based on everything before the first underscore
        location = file.split('_')[0]
        df['Location'] = location
    
    return concatenated_df

In [ ]:
all_traffic_df = import_and_concatenate_csvs('Data')

In [ ]:
# get times
all_traffic_df[['time', 'end_time']] = all_traffic_df['timeInterval'].str.split(' - ', expand=True)
all_traffic_df['time'] = pd.to_datetime(default_date + ' ' + all_traffic_df['time'])
# convert time to just time format
all_traffic_df['time'] = all_traffic_df['time'].dt.time

# get dates as datetime
all_traffic_df['date'] = pd.to_datetime(all_traffic_df['date'])
all_traffic_df['date'] = all_traffic_df['date'].dt.date

# drop and rename excess columns
all_traffic_df.drop(columns=['timeInterval', 'movement',  'end_time'], inplace=True)
all_traffic_df.rename(columns={'movementType': 'movement', 'count': 'volume', 'date':'date_count', 'time':'time_count'}, inplace=True)

all_traffic_df['class'] = all_traffic_df['class'].str.replace('_', ' ').str.title()
all_traffic_df['class'] = all_traffic_df['class'].replace(class_dict)

mask = all_traffic_df['movement'] == 'CROSSING'
all_traffic_df.loc[mask, 'approach'] = all_traffic_df.loc[mask, 'approach'].map(vru_approach_dict)

all_traffic_df

In [ ]:
all_traffic_df.to_csv('all_traffic.csv', index=False)

#### Load data into SDE

In [ ]:
# truncate and append to the database
def truncate_and_append_to_db(df, table_name, db_path):
    

#### Publish Map Service

In [ ]:
# Define the ArcGIS Portal (could be AGOL, Enterprise, or Server)
gis_url = 'https://maps.trpa.org/portal/home/'  # Change to your ArcGIS Portal URL (for AGOL or your enterprise portal)
username = os.environ.get('GIS_USERNAME')  
password = os.environ.get('GIS_PASSWORD')

# Connect to the GIS portal
gis = GIS(gis_url, username, password)

# Define the path to your ArcGIS Pro project (.aprx) and the feature layer you want to publish
aprx_path = r'C:\path\to\your\map.aprx'
feature_layer_name = 'your_feature_layer_name'  # Name of the feature layer you want to publish

# Connect to your ArcGIS Server or Portal for Publishing
server_url = 'https://your-arcgis-server-url/arcgis'  # Change to your ArcGIS Server URL
admin_user = 'admin_username'
admin_password = 'admin_password'

# Connect to ArcGIS Server using the Admin API
server = AdminAPI(url=server_url, username=admin_user, password=admin_password)

# Load the ArcGIS Pro project
aprx = arcpy.mp.ArcGISProject(aprx_path)
map_frame = aprx.listMaps()[0]  # Get the first map in the project

# Find the feature layer in the map
feature_layer = None
for layer in map_frame.listLayers():
    if layer.name == feature_layer_name:
        feature_layer = layer
        break

# Check if the feature layer was found
if feature_layer is None:
    raise ValueError(f"Feature layer '{feature_layer_name}' not found in the map.")

# Prepare to publish the feature layer as a Feature Service
def publish_feature_service(aprx, feature_layer, service_properties):
    # Convert the feature layer to a service definition draft
    sd_draft = feature_layer.getDefinition()

    # Set the properties for the service
    sd_draft.title = service_properties['Traffic Counts from ']
    sd_draft.snippet = service_properties['snippet']
    sd_draft.tags = service_properties['Traffic Counts by Location', 
                                       'Transportation', 'Traffic Counts',
                                       'TRPA']
    sd_draft.description = service_properties['description']
    
    # Define the service type (Feature Service)
    sd_draft.type = "Feature Service"

    # Now, publish the feature layer as a service
    try:
        print(f"Publishing feature layer '{feature_layer.name}' as a feature service...")
        feature_layer.publish()  # This step actually uploads the feature layer as a service
        print(f"Feature service '{service_properties['title']}' has been published successfully.")
    except Exception as e:
        print(f"Error publishing feature service: {e}")

# Define the service properties (modify these according to your needs)
service_properties = {
    'title': 'My Feature Service',
    'snippet': 'A feature service containing spatial data.',
    'tags': ['feature', 'service', 'geospatial'],
    'description': 'This feature service provides data on certain geographical entities.'
}

# Run the publish function
publish_feature_service(aprx, feature_layer, service_properties)
